# Predict binding affinity from a protein-ligand crystal structure

In [1]:
# Packages installed in my conda environment: Deepchem, rdkit, pytorch, pytorch-geometric, tensorflow, pdbfixer, numpy <1.25
# Deep Learning for the Life Sciences by Bharath Ramsundar, Peter Eastman, Patrick Walters and Vijay Pande
# pdbbind dataset: Cheng, T.J. et al. J. Chem. Inf. Model., 2009, 49, 1079-1093. (PDBbind v.2009)

In [5]:
import deepchem as dc

In [6]:
featurizer = dc.feat.RdkitGridFeaturizer(voxel_width=2.0,
                                         feature_types=["ecfp", "splif", "salt_bridge", "hbond"],
                                         flatten = True,
                                         sanitize = True)

In [7]:
import numpy as np
np.__version__

'1.24.4'

In [10]:
tasks, datasets, transformers = dc.molnet.load_pdbbind(featurizer=featurizer, reload=False, set_name='core' )

[10:08:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:08:33] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:08:34] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:08:35] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:08:35] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:08:36] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:08:37] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[10:08:37] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
/home/dbsejrgus226/miniconda3/envs/Deepchem/lib/python3.10/site-packages/deepchem/utils/geometry_utils.p

In [12]:
tasks

['-logKd/Ki']

In [13]:
transformers

In [14]:
train, vaild, test = datasets

In [15]:
print(train.X.dtype)
print(train.y.dtype)
print(train.w.dtype)

float64
float64
float32


In [16]:
x_train = train.X.astype("float32")
y_train = train.y.astype("float32")
w_train = train.w.astype("float32")

x_test = test.X.astype("float32")
y_test = test.y.astype("float32")
w_test = test.w.astype("float32")

In [17]:
x_train.dtype

dtype('float32')

In [19]:
train.X.shape

(154, 1)

In [ ]:
train_dataset = dc.data.NumpyDataset(x_train, y_train, w_train)
test_dataset = dc.data.NumpyDataset(x_test, y_test, w_test)

In [ ]:
train_dataset.X.shape

In [ ]:
model = dc.models.MultitaskRegressor(n_tasks=1, n_features=train_dataset.X.shape[1], layer_sizes=[5000, 2000, 1000], dropouts=0.5)

In [ ]:
model.fit(train_dataset, nb_epoch=200)

In [ ]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

In [ ]:
train_score = model.evaluate(train_dataset, [metric], transformers)
print(train_score)

In [ ]:
test_score = model.evaluate(test_dataset, [metric], transformers)
print(test_score)

In [ ]:
test_dataset.X.shape

In [ ]:
prediction = model.predict_on_batch(test_dataset.X)
prediction[:5]

In [ ]:
test_dataset.y[:5]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
prediction.shape

In [ ]:
prediction = prediction.reshape(20)
prediction.shape

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax = sns.scatterplot(x=test_dataset.y, y=prediction)
ax.set_title('pdbbind dataset')
ax.set_xlabel('True')
ax.set_ylabel('prediction')